In [ ]:
import pandas as pd
import deepxde as dde
import numpy as np

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
df=pd.read_csv("sars_2003_complete_dataset_clean.csv")
#... data arrangement goes here
#Finish with the order of: Suspected, Infected, Recovered
ndf=ndf.reindex(columns=['sus','infected','Number recovered'])
ndf=ndf.reindex(columns=['sus','infected','Number recovered'])

In [ ]:
print(ndf)

In [ ]:
#ndf['sus']=ndf['sus']/tot
#ndf.insert(loc=3, column='e', value=0)
#ndf['infected']=ndf['infected']/tot

In [ ]:
ndf=ndf.reindex(columns=['sus','infected','Number recovered','e'])
ndf['sus']=ndf['sus']/tot
ndf['infected']=ndf['infected']/tot
ndf['Number recovered']=ndf['Number recovered']/tot
tot=1
print(ndf)
narray=np.array(ndf)

In [ ]:
def gen_traindata():
    return np.arange(1,96+1),narray

In [ ]:
beta = dde.Variable(0.1)
gamma = dde.Variable(0.1)
eps = dde.Variable(0.1)

In [ ]:
def ode_system(x, y):
    SE,I,R,E = y[:, 0:1], y[:, 1:2], y[:,2:3], y[:,3:4]
    ds_x = dde.grad.jacobian(y, x, i=0)
    di_x = dde.grad.jacobian(y, x, i=1)
    dr_x = dde.grad.jacobian(y, x, i=2)
    de_x = dde.grad.jacobian(y, x, i=3)
    return [ds_x-eps*E,di_x-eps*E+(gamma)*I,dr_x-gamma*I,de_x-beta*I*(SE-E)/tot+eps*E,SE+I+R-tot]

In [ ]:
def boundary(_, on_initial):
    return on_initial

geom = dde.geometry.TimeDomain(0, 100)
ic1 = dde.icbc.IC(geom, lambda x: 0.968, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: 0.031, boundary, component=1)
ic3 = dde.icbc.IC(geom, lambda x: 0.001, boundary, component=2)

In [ ]:
def output_transform(t,y):
    y1 = y[:, 0:1]
    y2 = y[:, 1:2]
    y3 = y[:, 2:3]
    y4 = y[:, 3:4]
    return torch.cat([torch.abs(y1)+1e-6,torch.abs(y2)+1e-6,torch.abs(y3)+1e-6,torch.abs(y4)+1e-6],dim=1)

In [ ]:
layer_size = [1,20,80,256,40,4]
activation = "elu"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)
observe_t, ob_y = gen_traindata()
observe_SE = dde.icbc.PointSetBC(observe_t[:,None].astype(float), ob_y[:, 0:1].astype(float),component=0)
observe_I = dde.icbc.PointSetBC(observe_t[:,None].astype(float), ob_y[:, 1:2].astype(float), component=1)
observe_R = dde.icbc.PointSetBC(observe_t[:,None].astype(float), ob_y[:, 2:3].astype(float), component=2)

In [ ]:
data = dde.data.PDE(
    geom,
    ode_system,
    [ic1, ic2, ic3, observe_SE, observe_I, observe_R],
    num_domain=400,
    num_boundary=10,
    anchors=observe_t[:,None].astype(float),
)

In [ ]:
net = dde.nn.FNN(layer_size, activation, initializer)
net.apply_output_transform(output_transform)
model = dde.Model(data, net)
model.compile("adam", lr=0.001, external_trainable_variables=[beta, gamma,eps])
variable = dde.callbacks.VariableValue(
   [beta, gamma, eps], period=600, filename="variables.dat"
)
losshistory, train_state = model.train(iterations=50000,callbacks=[variable])

# train lbfgs

In [ ]:
print(beta,gamma,eps)

In [ ]:
model.compile("L-BFGS", external_trainable_variables=[beta, gamma,eps])
variable = dde.callbacks.VariableValue(
   [beta, gamma, eps], period=600, filename="variables.dat"
)
losshistory, train_state = model.train(callbacks=[variable])

In [ ]:
from deepxde.utils.external import *
def pack_data(train_state):
    def merge_values(values):
        if values is None:
            return None
        return np.hstack(values) if isinstance(values, (list, tuple)) else values

    y_train = merge_values(train_state.y_train)
    y_test = merge_values(train_state.y_test)
    best_y = merge_values(train_state.best_y)
    best_ystd = merge_values(train_state.best_ystd)
    print(best_y.shape)
    return y_train, y_test, best_y, best_ystd
s = pack_data(train_state)

In [ ]:
def myplot_best_state(train_state):
    if isinstance(train_state.X_train, (list, tuple)):
        print(
            "Error: The network has multiple inputs, and plotting such result han't been implemented."
        )
        return
    y_train, y_test, best_y, best_ystd = pack_data(train_state)
    y_dim = best_y.shape[1]
    # Regression plot
    # 1D
    if train_state.X_test.shape[1] == 1:
        idx = np.argsort(train_state.X_test[:, 0])
        X = train_state.X_test[idx, 0]
        plt.figure()
        for i in range(y_dim):
            if y_train is not None:
                plt.plot(train_state.X_train[:, 0], y_train[:, i], "ok", label="Train")
            if y_test is not None:
                plt.plot(X, y_test[idx, i], "-k", label="True")
            if i==0:
                plt.plot(X, best_y[idx, i]-best_y[idx, 3], color='red', label="S")
            elif i==1:
                plt.plot(X, best_y[idx, i], color='green', label="I")
            elif i==2:
                plt.plot(X, best_y[idx, i], color='blue', label="R")
            else:
                plt.plot(X, best_y[idx, i], color='purple', label="E")
            if best_ystd is not None:
                plt.plot(
                    X, best_y[idx, i] + 2 * best_ystd[idx, i], "-b", label="95% CI"
                )
                plt.plot(X, best_y[idx, i] - 2 * best_ystd[idx, i], "-b")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()


In [ ]:
myplot_best_state(train_state)